In [1]:
!pip install face_recognition
!pip install flask
!pip install flask-ngrok

In [ ]:
import face_recognition
from PIL import Image, ImageDraw
import numpy as np
import io
from flask import Flask, request, jsonify
import tempfile

# Initialize the Flask application
app = Flask(__name__)

# Known face encodings and corresponding names
known_faces = []

# Function to recognize faces and return names
def recognize_faces(image):
    face_names = []

    # Find all face locations and encodings in the uploaded image
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)

    for face_encoding in face_encodings:
        # Compare each face encoding to the known face encodings
        matches = face_recognition.compare_faces([known_face["encoding"] for known_face in known_faces], face_encoding)
        name = "Unknown"

        # If a match is found, use the known name
        if True in matches:
            first_match_index = matches.index(True)
            name = known_faces[first_match_index]["name"]

        face_names.append(name)

    return face_names

# Route for adding known faces
@app.route('/add_known_face', methods=['POST'])
def add_known_face():
    try:
        data = request.get_json()
        name = data.get('name')
        encoding_str = data.get('encoding')  # Get the encoding as a string

        if name and encoding_str is not None:
            # Convert the encoding string to a list of floats
            encoding = [float(value) for value in encoding_str.split(',')]
            
            known_faces.append({"name": name, "encoding": encoding})
            return jsonify({'message': 'Known face added successfully'}), 200
        else:
            return jsonify({'error': 'Invalid data format'}), 400

    except Exception as e:
        return jsonify({'error': str(e)}), 500


# Route for uploading an image and performing face recognition
@app.route('/upload', methods=['POST'])
def upload_image():
    try:
        # Get the uploaded file
        uploaded_file = request.files['image']

        if uploaded_file is None:
            return jsonify({'error': 'No file part'}), 400

        # Read the uploaded image
        img = face_recognition.load_image_file(uploaded_file)

        # Recognize faces and get their names
        face_names = recognize_faces(img)

        # Convert the image to a PIL Image for drawing
        pil_image = Image.fromarray(img)
        draw = ImageDraw.Draw(pil_image)

        # Draw rectangles and names around detected faces
        for (top, right, bottom, left), name in zip(face_recognition.face_locations(img), face_names):
            draw.rectangle(((left, top), (right, bottom)), outline=(0, 255, 0), width=2)
            draw.text((left, top - 15), name, fill=(0, 255, 0))

        # Save the result image temporarily
        with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as temp_file:
            result_image_path = temp_file.name
            pil_image.save(result_image_path)

        # Return the result file path and recognized names
        return jsonify({'result_image_path': result_image_path, 'recognized_names': face_names})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)  # Start ngrok when the app is run
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://d302-2001-448a-402d-1e86-d84f-36fb-6fbb-b53.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Oct/2023 19:48:25] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2023 19:48:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2023 19:53:26] "POST /add_known_face HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 19:53:52] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 19:57:04] "POST /add_known_face HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 19:57:17] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 19:58:35] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:02:36] "POST /add_known_face HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:02:53] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:04:06] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:04:12] "GET /upload HTTP/1.1" 405 -
127.0.0.1 - - [04/Oct/2023 20:04:37] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:05:51] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:06:46] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 20:08:42] "POST